In [ ]:
# %% [markdown]
# # GTWR+GNN — Experiments & OOS Comparison (Fixed)
# Robust to train_model return shape (dict/tuple). Adds GTWR baseline.

# %%
import os, math, warnings, random, inspect
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

warnings.filterwarnings("ignore")
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --- utilities ---
def safe_call(fn, /, *args, **kwargs):
    """Call fn with only kwargs it accepts (prevents unexpected-kw errors)."""
    sig = inspect.signature(fn)
    filt = {k:v for k,v in kwargs.items() if k in sig.parameters}
    return fn(*args, **filt)

def to_numpy(x):
    if torch.is_tensor(x):
        return x.detach().cpu().numpy()
    return np.asarray(x)

def local_ridge_wls(X_t, y_t, W_t, ridge_lambda=10.0, return_betas=False):
    """
    Per-row local ridge WLS: beta_i = argmin sum_j w_ij (y_j - x_j'beta)^2 + λ||beta||^2
    """
    X, y, W = X_t, y_t, W_t
    N, p = X.shape
    I = ridge_lambda * torch.eye(p, device=X.device, dtype=X.dtype)
    y_hat = torch.zeros(N, device=X.device, dtype=X.dtype)
    betas = torch.zeros(N, p, device=X.device, dtype=X.dtype) if return_betas else None
    for i in range(N):
        w = W[i]
        w_sqrt = torch.sqrt(w + 1e-12)
        Xw = X * w_sqrt.unsqueeze(1)
        yw = y * w_sqrt
        XtWX = Xw.t() @ Xw + I
        XtWy = Xw.t() @ yw
        try:
            beta = torch.linalg.solve(XtWX, XtWy)
        except RuntimeError:
            beta = torch.linalg.lstsq(XtWX, XtWy.unsqueeze(1)).solution.squeeze()
        y_hat[i] = X[i] @ beta
        if return_betas: betas[i] = beta
    return (y_hat, betas) if return_betas else y_hat

def metrics(y_true, y_pred):
    if y_true is None or y_pred is None or len(y_true)==0:
        return np.nan, np.nan, np.nan
    return (
        float(np.sqrt(mean_squared_error(y_true, y_pred))),
        float(mean_absolute_error(y_true, y_pred)),
        float(r2_score(y_true, y_pred))
    )

# --- package API ---
from gtwr_gnn import (
    load_panel_xlsx, build_panel_arrays, split_train_val_test, year_rows,
    MathematicallyCorrectGNNWeightNet, build_spatiotemporal_kernel,
    train_model, predict_new_fullgraph, predict_new_oos_transductive,
    finetune_transductive_with_future
)

# %% [markdown]
# ## 1) Load 2019–2022 (train/val/test), hold out 2023 as OOS

# %%
# Config columns / file
PATH_XLSX = "Data BPS Laporan KP - Coded.xlsx"
LAT_COL, LON_COL = "lat", "lon"
TIME_COL, TARGET_COL = "Tahun", "y"
FEATURE_COLS = ["X1","X2","X3","X4","X5","X6","X7","X8"]

df_full = load_panel_xlsx(PATH_XLSX, LAT_COL, LON_COL, TIME_COL, TARGET_COL, FEATURE_COLS)

df_1922 = df_full[df_full[TIME_COL] < 2023].copy()
df_2023 = df_full[df_full[TIME_COL] == 2023].copy().sort_values([LAT_COL, LON_COL]).reset_index(drop=True)

times_1922 = sorted(df_1922[TIME_COL].unique())
P = build_panel_arrays(df_1922, TIME_COL, TARGET_COL, FEATURE_COLS, LAT_COL, LON_COL, times_1922)
X_all, y_all = P['X_all'], P['y_all']
coords_blocks, times, N_per_year = P['coords_blocks'], P['times'], P['N_per_year']
coords_all = np.vstack(coords_blocks)
print("Years:", times, "| N_per_year:", N_per_year, "| X:", X_all.shape)

split = split_train_val_test(times, N_per_year, use_val=True)
train_rows, val_rows, test_rows = split['train_rows'], split['val_rows'], split['test_rows']
print(f"Split | Train={len(train_rows)}, Val={len(val_rows)}, Test={len(test_rows)}")

W_prior = build_spatiotemporal_kernel(coords_blocks, times, tau_s=1.0, tau_t=1.0,
                                      k_neighbors=8, prior_self_weight=1.0)
A_prior = torch.tensor(W_prior, dtype=torch.float32, device=device)

X_t = torch.tensor(X_all, dtype=torch.float32, device=device)
y_t = torch.tensor(y_all, dtype=torch.float32, device=device)
A_t = torch.tensor(W_prior, dtype=torch.float32, device=device)

# %% [markdown]
# ## 2) Baseline: GTWR-prior (no GNN)

# %%
y_gtwr = local_ridge_wls(X_t, y_t, A_t, ridge_lambda=10.0, return_betas=False)
y_gtwr_np = to_numpy(y_gtwr)

def split_metrics_np(pred_np):
    def f(rows):
        if rows is None or len(rows)==0: return (np.nan, np.nan, np.nan)
        return (
            float(np.sqrt(mean_squared_error(y_all[rows], pred_np[rows]))),
            float(mean_absolute_error(y_all[rows], pred_np[rows])),
            float(r2_score(y_all[rows], pred_np[rows])),
        )
    return f(train_rows) + f(val_rows) + f(test_rows)

rmse_tr, mae_tr, r2_tr, rmse_va, mae_va, r2_va, rmse_te, mae_te, r2_te = split_metrics_np(y_gtwr_np)
in_sample_rows = [dict(
    name="GTWR-prior (no-GNN)",
    rmse_tr=rmse_tr, mae_tr=mae_tr, r2_tr=r2_tr,
    rmse_va=rmse_va, mae_va=mae_va, r2_va=r2_va,
    rmse_te=rmse_te, mae_te=mae_te, r2_te=r2_te,
)]

# %% [markdown]
# ## 3) Train configs (robust unwrapping for train_model return)

# %%
def unwrap_train_output(res, model_seed, ridge_lambda=10.0):
    """
    Support:
      - dict: may have 'model', 'y_hat', 'history'
      - tuple: (model, history, (y_hat, ...)) or variants
      - fallback: recompute y_hat via current model & A_t
    """
    model_tr = model_seed
    y_hat_t = None
    history = []

    if isinstance(res, dict):
        model_tr = res.get('model', model_seed)
        y_hat_t = res.get('y_hat', None)
        history = res.get('history', [])
    elif isinstance(res, (tuple, list)):
        # try common pattern (model, history, (y_hat, betas, W, H))
        if len(res) >= 1 and hasattr(res[0], "state_dict"):
            model_tr = res[0]
        if len(res) >= 2:
            history = res[1]
        if len(res) >= 3:
            pack = res[2]
            if torch.is_tensor(pack) and pack.shape[0] == X_t.shape[0]:
                y_hat_t = pack
            elif isinstance(pack, (tuple, list)):
                for item in pack:
                    if torch.is_tensor(item) and item.shape[0] == X_t.shape[0]:
                        y_hat_t = item; break

    # fallback recompute predictions
    if y_hat_t is None:
        with torch.no_grad():
            W_learned, _ = model_tr(X_t, A_t)
            y_hat_t = local_ridge_wls(X_t, y_t, W_learned, ridge_lambda=ridge_lambda, return_betas=False)

    return model_tr, y_hat_t, history

def run_one_config(cfg_name,
                   wls_kind="ridge", ridge_lambda=10.0, huber_delta=1.0, huber_iters=5,
                   ent_w=5e-3, smooth_w=1e-3,
                   graph_topk=None, graph_symmetrize=False,
                   use_kl_to_prior=None, kl_weight=None, edge_dropout_p=None,
                   epochs=200, lr=1e-3, early_stop_patience=80, print_every=25):
    model = MathematicallyCorrectGNNWeightNet(d_in=X_all.shape[1]).to(device)
    res = safe_call(
        train_model,
        model=model,
        X_all=X_all, y_all=y_all, A_prior=A_prior,
        train_rows=train_rows, val_rows=val_rows, test_rows=test_rows,
        epochs=epochs, lr=lr, ridge_lambda=ridge_lambda,
        wls_kind=wls_kind, huber_delta=huber_delta, huber_iters=huber_iters,
        ent_w=ent_w, smooth_w=smooth_w, N_per_year=N_per_year, times=times,
        graph_topk=graph_topk, graph_symmetrize=graph_symmetrize,
        use_kl_to_prior=use_kl_to_prior, kl_weight=kl_weight,
        edge_dropout_p=edge_dropout_p,
        early_stop_patience=early_stop_patience, device=device, print_every=print_every
    )
    model_tr, y_hat_t, history = unwrap_train_output(res, model, ridge_lambda=ridge_lambda)
    y_pred = to_numpy(y_hat_t)

    def split(rows):
        if rows is None or len(rows)==0: return (np.nan, np.nan, np.nan)
        return (
            float(np.sqrt(mean_squared_error(y_all[rows], y_pred[rows]))),
            float(mean_absolute_error(y_all[rows], y_pred[rows])),
            float(r2_score(y_all[rows], y_pred[rows])),
        )

    sm = dict(
        name=cfg_name,
        rmse_tr=split(train_rows)[0], mae_tr=split(train_rows)[1], r2_tr=split(train_rows)[2],
        rmse_va=split(val_rows)[0],   mae_va=split(val_rows)[1],   r2_va=split(val_rows)[2],
        rmse_te=split(test_rows)[0],  mae_te=split(test_rows)[1],  r2_te=split(test_rows)[2],
    )
    return sm, model_tr, history

experiments = []

# B) GNN + Ridge WLS + entropy + TopK=12
summB, modelB, histB = run_one_config(
    "GNN+Ridge+Entropy+TopK12",
    wls_kind="ridge", ridge_lambda=10.0,
    ent_w=5e-3, smooth_w=1e-3, graph_topk=12, graph_symmetrize=False,
    epochs=200, lr=1e-3
)
experiments.append((summB, modelB))

# C) GNN + Huber WLS + TopK=12 + symmetrize
summC, modelC, histC = run_one_config(
    "GNN+Huber+TopK12+Sym",
    wls_kind="huber", ridge_lambda=10.0, huber_delta=1.0, huber_iters=5,
    ent_w=5e-3, smooth_w=1e-3, graph_topk=12, graph_symmetrize=True,
    epochs=200, lr=1e-3
)
experiments.append((summC, modelC))

# D) Optionally: KL-to-prior (ignored if train_model doesn't support)
summD, modelD, histD = run_one_config(
    "GNN+KLprior+TopK8",
    wls_kind="ridge", ridge_lambda=10.0,
    ent_w=0.0, smooth_w=1e-3, graph_topk=8, graph_symmetrize=False,
    use_kl_to_prior=True, kl_weight=1e-3,  # safely ignored if unsupported
    epochs=200, lr=1e-3
)
experiments.append((summD, modelD))

# E) GNN + Ridge + TopK=6
summE, modelE, histE = run_one_config(
    "GNN+Ridge+Entropy+TopK6",
    wls_kind="ridge", ridge_lambda=7.0,
    ent_w=5e-3, smooth_w=1e-3, graph_topk=6, graph_symmetrize=False,
    epochs=200, lr=1e-3
)
experiments.append((summE, modelE))

in_sample_df = pd.DataFrame(in_sample_rows + [e[0] for e in experiments])
print("== In-sample summary (2019–2022) ==")
display(in_sample_df)

# %% [markdown]
# ## 4) OOS 2023 — (a) Transductive only, (b) Fullgraph, (c) Retrain-Transductive

# %%
def eval_oos_for_model(model, cfg_name, cross_topk=12, lambda_blend=0.8, new_self_weight=0.0):
    """
    Compare three OOS strategies on 2023:
      a) OOS-Transductive (freeze old rows, new→old only)
      b) OOS-Fullgraph (semi-supervised: train+new in one graph, single forward)
      c) OOS-Retrain-Transductive (fine-tune with 2019–2023 graph; 2023 masked from loss)
    Returns a dict with metrics for all three.
    """
    if df_2023 is None or len(df_2023) == 0:
        return dict(
            name=cfg_name,
            rmse_oos_trans=np.nan, mae_oos_trans=np.nan, r2_oos_trans=np.nan,
            rmse_oos_full=np.nan,  mae_oos_full=np.nan,  r2_oos_full=np.nan,
            rmse_oos_reft=np.nan,  mae_oos_reft=np.nan,  r2_oos_reft=np.nan
        )

    # ---------- (a) OOS-Transductive ----------
    y_pred_oos_trans = safe_call(
        predict_new_oos_transductive,
        model=model,
        X_train=X_all, y_train=y_all,
        coords_train=coords_all, times_train=np.repeat(times, N_per_year),
        new_df=df_2023, feature_cols=FEATURE_COLS,
        time_col=TIME_COL, lat_col=LAT_COL, lon_col=LON_COL,
        tau_s=1.0, tau_t=1.0, knn_k=8, prior_self_weight=1.0,
        lambda_blend=lambda_blend, cross_topk=cross_topk, new_self_weight=new_self_weight,
        wls_kind="ridge", ridge_lambda=10.0, huber_delta=1.0, huber_iters=5,
        device=device
    )
    y_pred_oos_trans = to_numpy(y_pred_oos_trans)
    yt = df_2023[TARGET_COL].values.astype(np.float32)
    rmse_o1 = float(np.sqrt(mean_squared_error(yt, y_pred_oos_trans)))
    mae_o1  = float(mean_absolute_error(yt, y_pred_oos_trans))
    r2_o1   = float(r2_score(yt, y_pred_oos_trans))

    # ---------- (b) OOS-Fullgraph ----------
    y_pred_oos_full = safe_call(
        predict_new_fullgraph,
        model=model,
        X_train=X_all, y_train=y_all,
        coords_train=coords_all, times_train=np.repeat(times, N_per_year),
        new_df=df_2023, feature_cols=FEATURE_COLS,
        time_col=TIME_COL, lat_col=LAT_COL, lon_col=LON_COL,
        tau_s=1.0, tau_t=1.0, knn_k=8, prior_self_weight=1.0,
        wls_kind="ridge", ridge_lambda=10.0, huber_delta=1.0, huber_iters=5,
        device=device
    )
    y_pred_oos_full = to_numpy(y_pred_oos_full)
    rmse_o2 = float(np.sqrt(mean_squared_error(yt, y_pred_oos_full)))
    mae_o2  = float(mean_absolute_error(yt, y_pred_oos_full))
    r2_o2   = float(r2_score(yt, y_pred_oos_full))

    # ---------- (c) OOS-Retrain-Transductive (fine-tune 2019–2023) ----------
    df_1923 = df_full[df_full[TIME_COL] <= 2023].copy()
    times_1923 = sorted(df_1923[TIME_COL].unique())
    P23 = build_panel_arrays(df_1923, TIME_COL, TARGET_COL,
                             FEATURE_COLS, LAT_COL, LON_COL, times_1923)
    X_all_23, y_all_23 = P23['X_all'], P23['y_all']
    coords_blocks_23, times_23, N_per_year_23 = P23['coords_blocks'], P23['times'], P23['N_per_year']

    tr_rows_ft = np.concatenate([year_rows(times_23, N_per_year_23, y) for y in times_23 if y <= 2021])
    va_rows_ft = year_rows(times_23, N_per_year_23, 2022)
    fu_rows_ft = year_rows(times_23, N_per_year_23, 2023)

    # NOTE: this function in your package REQUIRES the 4 "full" arguments.
    ft = safe_call(
        finetune_transductive_with_future,
        model=model,
        # subset-for-loss (you already pass all 2019–2023 above, but keep naming per API)
        X_all=X_all_23, y_all=y_all_23,
        coords_blocks=coords_blocks_23, times=times_23,
        # REQUIRED "full" panel args (same objects in this use-case)
        X_all_full=X_all_23, y_all_full=y_all_23,
        coords_blocks_full=coords_blocks_23, times_full=times_23,
        # splits & hparams
        train_rows=tr_rows_ft, val_rows=va_rows_ft, future_rows=fu_rows_ft,
        lr=1e-4, epochs=150, ridge_lambda=10.0,
        ent_w=5e-3, smooth_w=1e-3,
        knn_k=8, tau_s=1.0, tau_t=1.0, prior_self_weight=1.0,
        N_per_year=N_per_year_23, print_every=25,
        wls_kind="ridge", graph_topk=None, graph_symmetrize=False,
        device=device
    )

    # Unwrap outputs (dict/tuple) → get y_hat for 2019–2023
    if isinstance(ft, dict):
        mdl_ft = ft.get('model', model)
        y_hat_23 = ft.get('y_hat', None)
    else:
        mdl_ft = model
        y_hat_23 = None
        if isinstance(ft, (tuple, list)) and len(ft) >= 1 and hasattr(ft[0], "state_dict"):
            mdl_ft = ft[0]
        if isinstance(ft, (tuple, list)) and len(ft) >= 3:
            pack = ft[2]
            if torch.is_tensor(pack) and pack.shape[0] == X_all_23.shape[0]:
                y_hat_23 = pack
            elif isinstance(pack, (tuple, list)):
                for item in pack:
                    if torch.is_tensor(item) and item.shape[0] == X_all_23.shape[0]:
                        y_hat_23 = item; break

    # Fallback: recompute y_hat_23 if not returned
    if y_hat_23 is None:
        with torch.no_grad():
            Wp = build_spatiotemporal_kernel(coords_blocks_23, times_23,
                                             tau_s=1.0, tau_t=1.0,
                                             k_neighbors=8, prior_self_weight=1.0)
            Ap = torch.tensor(Wp, dtype=torch.float32, device=device)
            Xt23 = torch.tensor(X_all_23, dtype=torch.float32, device=device)
            yt23 = torch.tensor(y_all_23, dtype=torch.float32, device=device)
            W_learned, _ = mdl_ft(Xt23, Ap)
            y_hat_23 = local_ridge_wls(Xt23, yt23, W_learned, ridge_lambda=10.0, return_betas=False)

    y23_pred = to_numpy(y_hat_23)
    rmse_o3 = float(np.sqrt(mean_squared_error(y_all_23[fu_rows_ft], y23_pred[fu_rows_ft])))
    mae_o3  = float(mean_absolute_error(y_all_23[fu_rows_ft], y23_pred[fu_rows_ft]))
    r2_o3   = float(r2_score(y_all_23[fu_rows_ft], y23_pred[fu_rows_ft]))

    return dict(
        name=cfg_name,
        rmse_oos_trans=rmse_o1, mae_oos_trans=mae_o1, r2_oos_trans=r2_o1,
        rmse_oos_full=rmse_o2,  mae_oos_full=mae_o2,  r2_oos_full=r2_o2,
        rmse_oos_reft=rmse_o3,  mae_oos_reft=mae_o3,  r2_oos_reft=r2_o3,
    )


# Evaluate OOS for trained models
oos_rows = []
for sm, mdl in experiments:
    oos_rows.append(eval_oos_for_model(mdl, sm['name'], cross_topk=12, lambda_blend=0.8, new_self_weight=0.0))
oos_df = pd.DataFrame(oos_rows)

# Merge in-sample + OOS
summary_all = in_sample_df.merge(oos_df, on='name', how='outer')
print("== Summary (Train/Val/Test + OOS types) ==")
display(summary_all)

# Save
os.makedirs("exp_outputs", exist_ok=True)
summary_all.to_csv("exp_outputs/gtwr_gnn_experiment_summary.csv", index=False)
print("Saved to exp_outputs/gtwr_gnn_experiment_summary.csv")


Device: cpu
Years: [2019, 2020, 2021, 2022] | N_per_year: 119 | X: (476, 8)
Split | Train=238, Val=119, Test=119
Building spatio-temporal kernel...
Time periods: 4, Locations(first): 119
Kernel construction complete. Sparsity: 0.981
Epoch   1 | Loss 0.8233 | RMSE: Train 0.911 | Val 0.999 | α 0.300 | τ 1.199
Epoch  25 | Loss 0.8232 | RMSE: Train 0.911 | Val 0.999 | α 0.305 | τ 1.171
Epoch  50 | Loss 0.8235 | RMSE: Train 0.911 | Val 0.999 | α 0.311 | τ 1.144
Epoch  75 | Loss 0.8234 | RMSE: Train 0.911 | Val 0.998 | α 0.316 | τ 1.119
Epoch 100 | Loss 0.8232 | RMSE: Train 0.911 | Val 0.998 | α 0.321 | τ 1.098
Epoch 125 | Loss 0.8230 | RMSE: Train 0.911 | Val 0.998 | α 0.327 | τ 1.079
Epoch 150 | Loss 0.8228 | RMSE: Train 0.911 | Val 0.998 | α 0.332 | τ 1.063
Epoch 175 | Loss 0.8226 | RMSE: Train 0.911 | Val 0.998 | α 0.337 | τ 1.049
Epoch 200 | Loss 0.8225 | RMSE: Train 0.911 | Val 0.998 | α 0.343 | τ 1.037
Epoch   1 | Loss 0.9033 | RMSE: Train 0.947 | Val 1.053 | α 0.300 | τ 1.199
Epoch  

,name,rmse_tr,mae_tr,r2_tr,rmse_va,mae_va,r2_va,rmse_te,mae_te,r2_te
0,GTWR-prior (no-GNN),0.904653,0.699241,0.895449,0.991236,0.758680,0.860982,1.093719,0.840128,0.767629
1,GNN+Ridge+Entropy+TopK12,0.910645,0.706778,0.894060,0.997990,0.766513,0.859081,1.099677,0.845533,0.765091
2,GNN+Huber+TopK12+Sym,0.946318,0.694693,0.885597,1.052727,0.771157,0.843199,1.161066,0.858116,0.738131
3,GNN+KLprior+TopK8,0.880614,0.684666,0.900932,0.977706,0.759201,0.864751,1.033214,0.783789,0.792628
4,GNN+Ridge+Entropy+TopK6,0.728979,0.550809,0.932112,0.834257,0.644318,0.901527,0.847853,0.650356,0.860360


Ep   1 | Loss 0.8968 | RMSE: Train 0.951 | Val 1.081 | Fut 0.879 | α=0.340, τ=1.044
Ep  25 | Loss 0.8966 | RMSE: Train 0.951 | Val 1.081 | Fut 0.879 | α=0.340, τ=1.041
Ep  50 | Loss 0.8963 | RMSE: Train 0.950 | Val 1.081 | Fut 0.879 | α=0.341, τ=1.039
Ep  75 | Loss 0.8961 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.341, τ=1.036
Ep 100 | Loss 0.8958 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.342, τ=1.034
Ep 125 | Loss 0.8956 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.342, τ=1.032
Ep 150 | Loss 0.8954 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.343, τ=1.029
Ep   1 | Loss 0.8961 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.341, τ=1.038
Ep  25 | Loss 0.8959 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.342, τ=1.035
Ep  50 | Loss 0.8956 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.342, τ=1.033
Ep  75 | Loss 0.8954 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.343, τ=1.030
Ep 100 | Loss 0.8952 | RMSE: Train 0.950 | Val 1.080 | Fut 0.879 | α=0.343, 

,name,rmse_tr,mae_tr,r2_tr,rmse_va,mae_va,r2_va,rmse_te,mae_te,r2_te,rmse_oos_trans,mae_oos_trans,r2_oos_trans,rmse_oos_full,mae_oos_full,r2_oos_full,rmse_oos_reft,mae_oos_reft,r2_oos_reft
0,GNN+Huber+TopK12+Sym,0.946318,0.694693,0.885597,1.052727,0.771157,0.843199,1.161066,0.858116,0.738131,1.347372,1.089169,0.552837,5.805849,5.432977,-7.302755,0.878877,0.669092,0.809740
1,GNN+KLprior+TopK8,0.880614,0.684666,0.900932,0.977706,0.759201,0.864751,1.033214,0.783789,0.792628,1.342107,1.084699,0.556325,5.718073,5.343064,-7.053602,0.883264,0.676801,0.807836
2,GNN+Ridge+Entropy+TopK12,0.910645,0.706778,0.894060,0.997990,0.766513,0.859081,1.099677,0.845533,0.765091,1.347214,1.089028,0.552942,5.803629,5.430682,-7.296406,0.878953,0.669317,0.809707
3,GNN+Ridge+Entropy+TopK6,0.728979,0.550809,0.932112,0.834257,0.644318,0.901527,0.847853,0.650356,0.860360,1.344106,1.086350,0.555002,5.756072,5.381886,-7.160995,0.881072,0.673687,0.808789
4,GTWR-prior (no-GNN),0.904653,0.699241,0.895449,0.991236,0.758680,0.860982,1.093719,0.840128,0.767629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Saved to exp_outputs/gtwr_gnn_experiment_summary.csv
